In [1]:
import pandas as pd
import numpy as np
import os
import Memory_Collaborative_Filtering as mem
import sqlite3 as sql
import sklearn 
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
import pickle

In [2]:
path = 'C:/Users/arjun/Desktop/amazon.db'
path2 = 'C:/Users/arjun/Desktop/frequency_rankings.db'

In [3]:
review_data = mem.import_standard_data(path)
review_data = review_data.drop_duplicates(['customer_id', 'product_id'])
review_data['star_rating'] = review_data['star_rating'].astype(int)
review_data['review_length'] = review_data['review_body'].str.count(' ') + 1

NameError: name 'user_counts_top_10000' is not defined

In [5]:
top_user_counts = mem.import_user_frequency_data(path2)

In [14]:
check = pd.read_csv('C:/users/arjun/Desktop/test.csv')

In [7]:
review_data = pd.merge(review_data, top_user_counts, left_on='customer_id', right_on='customer_id', how='inner')

In [15]:
review_data2 = pd.merge(review_data, check, left_on='product_id', right_on='product_id', how='left')

In [8]:
final = review_data[['star_rating', 'review_length', 'counts']]

In [9]:
nbrs = NearestNeighbors(n_neighbors=6, algorithm = 'ball_tree').fit(final)

In [10]:
def get_index_from_name(name):
    return review_data[review_data["product_title"]==name].index.tolist()[0]

In [11]:
def get_id_from_partial_name(partial):
    for name in all_anime_names:
        if partial in name:
            print(name,all_anime_names.index(name))

In [12]:
def print_similar_books(query=None,id=None, model=None):
    distances2, indices2 = model.kneighbors()
    if id:
        for id in indices2[id][1:]:
            print(review_data.iloc[id]["product_title"])
    if query:
        found_id = get_index_from_name(query)
        for id in indices2[found_id][1:]:
            print(review_data.iloc[id]["product_title"])

In [13]:
print_similar_books(query="The Choice", model=nbrs)

Waiting On You (Blue Heron)
Istanbul Passage: A Novel
The Innocent: A Novel
The Witch with No Name
How Children Succeed: Grit, Curiosity, and the Hidden Power of Character


In [ ]:
filename = 'finalized_model.sav'
pickle.dump(nbrs, open(filename, 'wb'))

In [ ]:
print(review_data['product_title'].head(10000))